 # SimpleGeo.py
<hr style="border:2px solid #0077b9;">

## Repositório GIT
<hr style="border:2px solid #0077b9;">
<img src="./imgs/simplegeo.git.png" style="width: 1200px;"/>
URL: [github.com/e-sensing/simple_geo.py](github.com/e-sensing/simple_geo.py)

## Usando a SimpleGeo.py
<hr style="border:2px solid #0077b9;">

### Listando Features e Coverages disponíveis
<hr style="border:2px solid #0077b9;">

In [ ]:
from SimpleGeo import SimpleGeo

# connect to wfs and wtss servers
s = SimpleGeo(wfs="http://www.terrama2.dpi.inpe.br/geoserver", wtss="http://www.terrama2.dpi.inpe.br/e-sensing")

# print available features
print("Features:")
for f in s.features()['features']:
    print("\t{}".format(f))

# print available coverages
print("\nCoverages:")
for c in s.coverages()['coverages']:
    print("\t{}".format(c))

## Obtendo Metadados das Features e Coverages
<hr style="border:2px solid #0077b9;">

In [ ]:
# Feature estados_bra
f = s.feature('esensing:estados_bra')

metadata = f.describe()

#print(metadata)

print("Nome: {}".format(metadata['full_name']))
print("Tipo: {}".format(metadata['geometry']['localtype']))

print("Atributos:")
for att in metadata['attributes']:
    print("\t{:<12}: {}".format(att['name'], att['localtype']))

In [ ]:
# Coverage rpth
c = s.coverage('rpth')

metadata = c.describe()
print("Nome: {}".format(metadata['name']))
print("Proj4: {}".format(metadata['crs']['proj4']))
print("Timeline: {} ... {}".format(metadata['timeline'][1], metadata['timeline'][-1]))
print("Atributos:")
for att in metadata['attributes']:
    print("\t{}".format(att))

#print(metadata)

## Obtendo dados de Features
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra')
estados = f.get()
# displaying 
display(estados.head(n=10))

## Métodos da SimpleGeo para Features
<hr style="border:2px solid #0077b9;">

### Métodos para Features

- **attributes**: define quais atributos devem ser retornados
- **max_features**: limita o número de features a serem retornadas
- **sort_by**: ordena os elementos por um atributo não espacial
- **filter**: seleciona por um ou mais atributos (espaciais ou não)



### Selecionando atributos de Features (attributes)
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra').attributes(['nome','regiao'])
estados = f.get()
# displaying 
display(estados.head(n=10))

### Limitando o número de retornos (max_features)
<hr style="border:2px solid #0077b9;">

In [ ]:
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5)
estados = f.get()
# displaying
display(estados)

### Ordenando features (sort_by)
<hr style="border:2px solid #0077b9;">

In [ ]:
# Ascendente ASC
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5).sort_by('nome')
estados = f.get()
# displaying
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

# Descesdente DESC
f = s.feature('esensing:estados_bra').attributes(['nome','regiao']).max_features(5).sort_by(pre.DESC('nome'))
estados = f.get()
# displaying
display(estados)

### Filtrando Features
<hr style="border:2px solid #0077b9;">

A **SimpleGeo** fornece os seguintes predicados para auxiliar a construção de um filtro:

- Atributos não espaciais:
    - EQ: igual
    - NE: diferente
    - LT: menor que
    - GT: maior que
    - LE: menor ou igual a
    - GE: maior ou igual a
    - AND: e
    - OR: ou

- Atributos espaciais:
    - WITHIN: dentro
    - INTERSECTS: intersecta

In [ ]:
# Only South region
f = s.feature('esensing:estados_bra')\
     .filter(pre.EQ('regiao','S'))
estados = f.get()
# displaying 
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

# South or North regions
f = s.feature('esensing:estados_bra')\
     .filter(pre.OR(
             pre.EQ('regiao','S'),
             pre.EQ('regiao','N')
         ))
estados = f.get()
# displaying 
display(estados)

In [ ]:
from SimpleGeo import Predicates as pre

f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .filter(pre.OR(
    pre.AND(
        pre.EQ("satelite_referencia", "true"),
        pre.GE("timestamp", "2016-01-01"),
        pre.LT("timestamp", "2016-02-01"),
        pre.WITHIN(
            "POLYGON((-49.515628859948507 -19.394602563415745,-48.020567850467053 -19.610579617637825,-48.354439522883652 -21.052347219666608,-49.849500507163917 -20.836369963642884,-49.515628859948507 -19.394602563415745))"),
    ), \
    pre.NE("regiao", "SE") \
    )) \
    .max_features(10) \
    .sort_by(pre.DESC("municipio"))
    
focos = f.get()
display(focos)

### Cache
<hr style="border:2px solid #0077b9;">


In [ ]:
from SimpleGeo import Predicates as pre
import time

# connect to wfs and wtss servers
s = SimpleGeo(wfs="http://www.terrama2.dpi.inpe.br/geoserver", wtss="http://www.terrama2.dpi.inpe.br/e-sensing", cache=True, cache_dir="/tmp/.cache")
#s.clear_cache()
f = s.feature("esensing:focos_bra_2016") \
    .attributes(["id", "municipio", "timestamp", "regiao"]) \
    .filter(pre.OR(
    pre.AND(
        pre.EQ("satelite_referencia", "true"),
        pre.GE("timestamp", "2016-01-01"),
        pre.LT("timestamp", "2016-02-01"),
        pre.WITHIN(
            "POLYGON((-49.515628859948507 -19.394602563415745,-48.020567850467053 -19.610579617637825,-48.354439522883652 -21.052347219666608,-49.849500507163917 -20.836369963642884,-49.515628859948507 -19.394602563415745))"),
    ), \
    pre.NE("regiao", "SE") \
    )) \
    .max_features(10) \
    .sort_by(pre.DESC("municipio"))

start = time.time()
focos = f.get()
end = time.time()

#display(focos)

# total time
print("Elapsed time: {}s".format(end-start))